## Dependencies

In [129]:
import discord
from discord.ext import commands

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import math

from datetime import datetime, timedelta  
import io
import os

# Load necessary tokens and IDs from .env file (not included in repository)
from dotenv import load_dotenv
_ = load_dotenv()

## Bot Setup

In [130]:
# All commands for bot will be prefixed with a period (e.g. '.help')
client = commands.Bot(command_prefix = '.')

# Running list of winning movie titles for the current week
titles = []

token = os.environ["DISCORD_TOKEN"]
channel_ID = int(os.environ["DISCORD_CHANNEL"])

In [131]:
# Message bot will print to console when it is connected and ready to receive commands
@client.event 
async def on_ready():
    print('Bot is ready. Use kill command to log bot out.')

## Available Commands

In [132]:
client.remove_command("tally")
@client.command(brief='Tally votes', description='Generates a bar chart of votes for all movies that received at least one reaction since last Saturday at 9:30 (UTC time)')
async def tally(ctx):
    await ctx.send("Tabulating votes...")
    
    channel = client.get_channel(channel_ID)
    
    # Votes should be pulled from the most recent Saturday at 9:30 EST
    
    today = datetime.utcnow()
    idx = (today.weekday() + 1) % 7
    lastSaturday = today - timedelta(7+idx-6)
    lastSaturday = lastSaturday.replace(hour=21, minute=30, second = 0)
    
    
    # Add movies and number of reactions (i.e. votes) and sort in descending order
    
    votes = []
    
    async for message in channel.history(after=lastSaturday):
        if len(message.reactions) > 0 and message.content not in titles:
            votes.append((message.content, len(message.reactions)))
            
    votes = pd.DataFrame.from_records(np.array(votes), columns = ['Movie', 'Number of Votes'])
    votes["Number of Votes"] = pd.to_numeric(votes["Number of Votes"])
    votes.sort_values(by = "Number of Votes", ascending = False, inplace = True)
    
    # Create horizontal bar chart of movie rankings
    
    rcParams.update({'figure.autolayout': True})
    rcParams.update({'figure.figsize': [16,9]})
    
    fig, ax = plt.subplots()

    movies = votes["Movie"]
    movies_range = np.arange(len(movies))
    ranking = votes["Number of Votes"]

    ax.barh(movies_range, ranking, align='center')
    ax.set_yticks(movies_range)
    ax.set_yticklabels(movies)
    ax.set_xticks(np.arange(math.ceil(max(ranking))+1))
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_xlabel('Number of Votes')
    ax.set_title('Clumsy Movie Ranking')

    # Save figure locally and then embed into message 
    
    fig.savefig('discord-images/graph.png')
    
    with open('discord-images/graph.png', 'rb') as f:
        file = io.BytesIO(f.read())    
    
    image = discord.File(file, filename='graph.png')
    embed = discord.Embed(title = "Votes as of " + datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
    embed.set_image(url=f'attachment://graph.png')
    
    await ctx.send("Votes:\n")
    await ctx.send(file=image, embed=embed)  


In [133]:
client.remove_command("wheel")
@client.command(brief='Prepare votes for the wheel', description='Generates a list for all movies that received at least one reaction since last Saturday at 9:30 (UTC time). Movie titles are duplicated according to number of votes.')
async def wheel(ctx):

    channel = client.get_channel(channel_ID)

    # Votes should be pulled from the most recent Saturday at 9:30 EST    
    
    today = datetime.utcnow()
    idx = (today.weekday() + 1) % 7
    lastSaturday = today - timedelta(7+idx-6)
    lastSaturday = lastSaturday.replace(hour=21, minute=30, second = 0)

    # Create a text list of all movie titles, copied according to number of votes
    
    wheel_list = ""
    
    async for message in channel.history(after=lastSaturday):
        if len(message.reactions) > 0 and message.content not in titles:
            wheel_list = wheel_list + (message.content + "\n") * len(message.reactions)
             
    await ctx.send("Wheel List:\n")
    await ctx.send(wheel_list)    

In [134]:
client.remove_command("kill")
@client.command(brief='Force logout for bot', description='Forces the bot to logoff Discord. Convenience function to interrupt process from jupyter notebook')
async def kill(ctx):
    await ctx.send("Thank you for using Clumsy Movie Bot. Goodbye.")
    
    # Log bot out of Discord
    await client.logout()
    
    # Clear internal cache of bot and prepare it to be reopened if necessary
    client.clear()

In [135]:
client.remove_command("purge")
@client.command(brief='Delete all messages', description='Removes last 1000 messages before current datetime (UTC)')
async def purge(ctx):
    channel = client.get_channel(channel_ID)
    
    # Removes the last 1000 messages in channel
    await channel.purge(limit = 1000, before = datetime.utcnow() + timedelta(1))

In [136]:
client.remove_command("samples")
@client.command(brief='Print 5 sample movies', description='Prints 5 seperate messages with a movie name. Reactions should be added to movie title to register vote.')
async def samples(ctx):
    
    # Create sample movie nominations with emoji reactions to simulate votes
    
    m1 = await ctx.send("Lair of the White Worm") 
    m2 = await ctx.send("Hausu") 
    m3 = await ctx.send("Hackers") 
    m4 = await ctx.send("Earth Girls are Easy") 
    m5 = await ctx.send("50 Shades Darker") 
    
    await m1.add_reaction('\U0001f44d')
    await m2.add_reaction('\U0001f44d')
    await m3.add_reaction('\U0001f44d')
    await m4.add_reaction('\U0001f44d')
    await m5.add_reaction('\U0001f44d')
    
    await m4.add_reaction('\U0001f600')
    await m5.add_reaction('\U0001f600')
    
    await m4.add_reaction('\U0001f603')

In [137]:
client.remove_command("winner")
@client.command(brief='Added winning movie to winner list', description='Add winning movie for the current week to list of winners. Run prior to rollover function.')
async def winner(ctx, *, title: str):
    
    titles.append(title)
    await ctx.send("Added winner: " + title)

In [138]:
client.remove_command("winner_list")
@client.command(brief='List winners', description='')
async def winner_list(ctx):
    
    await ctx.send(titles)

In [142]:
client.remove_command("rollover")
@client.command(brief='Create a rollover list', description='Create a rollover list for the next week, with movies that have at least 1 vote. NOTE: Add winners to winner list first with winner command')
async def rollover(ctx):
    
    await ctx.send("Next Week on the Wheel:")
    
    channel = client.get_channel(channel_ID)
    
    # Votes should be pulled from the most recent Saturday at 9:30 EST
    
    today = datetime.utcnow()
    idx = (today.weekday() + 1) % 7
    lastSaturday = today - timedelta(7+idx-6)
    lastSaturday = lastSaturday.replace(hour=21, minute=30, second = 0)    
    
    async for message in channel.history(after=lastSaturday):
        if len(message.reactions) > 0 and message.content not in titles:
            await ctx.send(message.content)  

## Connect Bot to Discord Server

If bot has not connected since computer startup, connect to discord:

In [139]:
await client.login(token, bot = True)

Connect bot to receive commands. Process will continue to run until interrupted in discord with **.kill** command

In [ ]:
await client.connect(reconnect = True)

Bot is ready. Use kill command to log bot out.


In [143]:
titles = []